# Get 3RWW Calibrated Radar Rainfall Data from the Teragon Rainfall API

> *This notebook is a work-in-progress*

*Get timeseries rainfall data for an area of interest within Allegheny County*

---

When radar estimates of rainfall are calibrated with actual rain gauge data, a highly accurate and valuable source of rainfall data can be calculated over large geographic areas. The result is called *Calibrated Radar Rainfall Data*, or *Gauge-Adjusted Radar Rainfall Data (GARRD)*

3 Rivers Wet Weather (3RWW), with support from [Vieux Associates](http://www.vieuxinc.com/), uses calibrated data from the NEXRAD radar located in Moon Township, PA with rain gauge measurements collected during the same time period and rain event for every square kilometer in Allegheny County. The resulting rainfall data is equivalent in accuracy to having 2,276 rain gauges placed across the County.

You can view and explore this data on 3RWW's calibrated radar rainfall data site at [www.3riverswetweather.org/municipalities/calibrated-radar-rainfall-data](http://www.3riverswetweather.org/municipalities/calibrated-radar-rainfall-data)

This notebook walks through how to programmatically access 3RWW's massive repository of high resolution spatiotemporal rainfall data for Allegheny County via the ***Teragon Rainfall Dataset API*** for an area of interest. The Teragon Rainfall Dataset API accesses a cache of the historic rainfall data that has been processed by Vieux Associates. It is the same data you'll find on 3RWW's [calibrated-radar-rainfall-data](http://www.3riverswetweather.org/municipalities/calibrated-radar-rainfall-data) website (which is also managed by Teragon).

Complete documentation for the Teragon Rainfall Dataset API is available at [3rww.github.io/api-docs](https://3rww.github.io/api-docs/?language=Python#teragon-rainfall-dataset-api-10).

## First: Notebook Setup

~~This assumes you've got things set up following the recommendations in the ***Getting Started*** notebook.~~

In [1]:
# imports from the Python standard library
import json #read and write JSON

We're going to use a few external Python packages to make our lives easier:

In [2]:
# imports from 3rd-party libraries
# Requests - HTTP requests for humans
import requests
# PETL - an Extract/Transform/Load toolbox
import petl as etl
# sortedcontainers provides a way to have sorted dictionaries (before Python 3.7)
from sortedcontainers import SortedDict
# Python DateUtil (parser) - a helper for reading timestamps
from dateutil.parser import parse
# ArcGIS API for Python - for accessing 3RWW's open reference datasets in ArcGIS Online
from arcgis.gis import GIS
from arcgis import geometry
# for displaying things from the ArcGIS Online in this Jupyter notebook
from IPython.display import display

## 1. The basics of getting calibrated radar rainfall data

Getting rainfall data programmatically is a mostly straightforward endeavor: it requires you to submit a HTTP request with parameters specifying locations of interest and a time range. It returns a `csv`-like plain-text response where time intervals are on the x-axis, locations are the y-axis, and values are rainfall amounts. Data quality metadata is included. 

Complete API documentation is available at [3rww.github.io/api-docs](https://3rww.github.io/api-docs/?language=Python#teragon-rainfall-dataset-api-10).

To demonstrate the basics of making a call to the API, we'll first use some pre-selected pixel values; we'll demonstrate how to get pixel locations from geodata later on, and then revisit submitting the request for specific locations.


### Assemble the request payload

We'll use the Python `requests` library to make our calls to the API.

The Hurricane Ivan rainfall event in 2004 will server as an example (2004-09-17 03:00 to 2004-09-18 00:00); the pixels used are pre-selected for this example (we'll get to identifying pixels for an area of interest in a bit).

The request payload for that event, as a Python `dictionary` looks like this:

In [3]:
data = {
    'startyear': 2004,
    'startmonth': 9,
    'startday': 17,
    'starthour': 3,
    'endyear': 2004,
    'endmonth': 9,
    'endday': 18,
    'endhour': 0,
    'interval': 'Hourly',
    'zerofill': 'yes',
    'pixels': '148,134;149,134;148,133;149,133'
}

The Teragon Rainfall Dataset API only accesspts `POST` requests. Using the Python `requests` library, then, we construct our call like this, using the `post` method:

In [4]:
response = requests.post(
    url="http://web.3riverswetweather.org/trp:API.pixel",
    data=data
)
# Note that the `data` argument for `request.post` is explicitly 
# used here: the API does not accept the request payload as 
# query string parameters (`params`), which is the default behavior 
# for the requests library.

That's it. The data is contained in the `response` variable. As mentioned earlier, the API returns a `csv`-like plain-text response where time intervals are on the x-axis, locations are the y-axis, and values are rainfall amounts. You can print the response:

In [5]:
print(response.text)

Timestamp,148-134,,149-134,,148-133,,149-133,
2004/09/17 03:00:00,0.0000,-,0.0000,-,0.0000,-,0.0000,-
2004/09/17 04:00:00,0.0040,-,0.0010,-,0.0050,-,0.0020,-
2004/09/17 05:00:00,0.0030,-,0.0030,-,0.0100,-,0.0030,-
2004/09/17 06:00:00,0.0430,-,0.0400,-,0.0450,-,0.0410,-
2004/09/17 07:00:00,0.1370,-,0.1389,-,0.1300,-,0.1430,-
2004/09/17 08:00:00,0.1090,-,0.1140,-,0.1040,-,0.1080,-
2004/09/17 09:00:00,0.2160,-,0.2200,-,0.1770,-,0.2089,-
2004/09/17 10:00:00,0.4260,-,0.3669,-,0.4610,-,0.4109,-
2004/09/17 11:00:00,0.4520,-,0.4100,-,0.4700,-,0.4560,-
2004/09/17 12:00:00,0.5590,-,0.6290,-,0.4390,-,0.4760,-
2004/09/17 13:00:00,1.4609,-,1.0550,-,0.9660,-,0.6470,-
2004/09/17 14:00:00,1.0889,-,1.1510,-,1.0340,-,1.0679,-
2004/09/17 15:00:00,0.7379,-,0.8900,-,0.6890,-,0.8200,-
2004/09/17 16:00:00,0.9260,-,0.8760,-,0.9860,-,0.8720,-
2004/09/17 17:00:00,0.3690,-,0.3609,-,0.3480,-,0.3409,-
2004/09/17 18:00:00,0.1948,-,0.1950,-,0.1669,-,0.1968,-
2004/09/17 19:00:00,0.1940,-,0.1810,-,0.1700,-,0.1740,-
20

## 2. Working with the data from the API

That raw response was a little hard to read, so we'll use the wonderful Python `PETL` library to get something human-readable (you might just as easily swap in the Python Pandas library to do this sort of thing).

In [6]:
table = etl.fromcsv(etl.MemorySource(response.text.encode()))
etl.vis.displayall(table)

Timestamp,148-134,,149-134,,148-133,,149-133,
2004/09/17 03:00:00,0.0000,-,0.0000,-,0.0000,-,0.0000,-
2004/09/17 04:00:00,0.0040,-,0.0010,-,0.0050,-,0.0020,-
2004/09/17 05:00:00,0.0030,-,0.0030,-,0.0100,-,0.0030,-
2004/09/17 06:00:00,0.0430,-,0.0400,-,0.0450,-,0.0410,-
2004/09/17 07:00:00,0.1370,-,0.1389,-,0.1300,-,0.1430,-
2004/09/17 08:00:00,0.1090,-,0.1140,-,0.1040,-,0.1080,-
2004/09/17 09:00:00,0.2160,-,0.2200,-,0.1770,-,0.2089,-
2004/09/17 10:00:00,0.4260,-,0.3669,-,0.4610,-,0.4109,-
2004/09/17 11:00:00,0.4520,-,0.4100,-,0.4700,-,0.4560,-
2004/09/17 12:00:00,0.5590,-,0.6290,-,0.4390,-,0.4760,-


That's better. Note that each pixel column has a column that follows it: the API response includes data quality metadata for every value if it exists. In this example, there isn't any data quality issues noted, thus the `-` following every value.

Once you've noted any data quality issues, you might consider removing those additional columns and clean things up to make working with the data a bit simpler, as follows:

In [7]:
def clean_up_response(response_text):
    """a helper function for cleaning up the API response, using PETL
    
    params:
    response_text = a Python `requests` library `response.text` object
    
    returns:
    a PETL table object
    """
    table = etl.fromcsv(etl.MemorySource(response_text.encode()))
    # get a list of the existing table header values
    h = list(etl.header(table))
    # we know every other column is a notes column, so we identify those
    xy_cols = zip(*[iter(h[1:])] * 2)
    # store some things:
    new_header = ['Timestamp']
    fields_to_cut = []
    # then we iterate over those columns and
    for each in xy_cols:
        # note the correct id, assembled from columns
        id_col, note_col = each[0], each[1]
        # assemble a new notes column name
        notes_col = "{0}-n".format(id_col)
        # add those to our new header (array)
        new_header.extend([id_col, notes_col])
        # track fields that we might want to remove
        fields_to_cut.append(notes_col)

    # transform the table
    table_cleaned = etl \
        .setheader(table, new_header) \
        .select('Timestamp', lambda v: v.upper() != 'TOTAL')  \
        .convert('Timestamp', lambda t: parse(t).isoformat())  \
        .replaceall('N/D', None)\
        .cutout(*tuple(fields_to_cut))

    return table_cleaned

In [8]:
table_cleaned = clean_up_response(response.text)

etl.vis.displayall(table_cleaned)

Timestamp,148-134,149-134,148-133,149-133
2004-09-17T03:00:00,0.0000,0.0000,0.0000,0.0000
2004-09-17T04:00:00,0.0040,0.0010,0.0050,0.0020
2004-09-17T05:00:00,0.0030,0.0030,0.0100,0.0030
2004-09-17T06:00:00,0.0430,0.0400,0.0450,0.0410
2004-09-17T07:00:00,0.1370,0.1389,0.1300,0.1430
2004-09-17T08:00:00,0.1090,0.1140,0.1040,0.1080
2004-09-17T09:00:00,0.2160,0.2200,0.1770,0.2089
2004-09-17T10:00:00,0.4260,0.3669,0.4610,0.4109
2004-09-17T11:00:00,0.4520,0.4100,0.4700,0.4560
2004-09-17T12:00:00,0.5590,0.6290,0.4390,0.4760


There it is. Export that to CSV with PETL like this:

```python
etl.tocsv(table_cleaned, "path/to/save/your/data.csv")
```

Now what if we want to work with this a key-value store? Try this:

In [9]:
data = SortedDict()
for row in etl.transpose(table_cleaned).dicts():
    inside = SortedDict()
    for d in row.items():
        if d[0] != 'Timestamp':
            if d[1]:
                v = float(d[1])
            else:
                v = d[1]
            inside[d[0]] = v
    data[row['Timestamp']] = inside
print(json.dumps(data, indent=2))

{
  "148-133": {
    "2004-09-17T03:00:00": 0.0,
    "2004-09-17T04:00:00": 0.005,
    "2004-09-17T05:00:00": 0.01,
    "2004-09-17T06:00:00": 0.045,
    "2004-09-17T07:00:00": 0.13,
    "2004-09-17T08:00:00": 0.104,
    "2004-09-17T09:00:00": 0.177,
    "2004-09-17T10:00:00": 0.461,
    "2004-09-17T11:00:00": 0.47,
    "2004-09-17T12:00:00": 0.439,
    "2004-09-17T13:00:00": 0.966,
    "2004-09-17T14:00:00": 1.034,
    "2004-09-17T15:00:00": 0.689,
    "2004-09-17T16:00:00": 0.986,
    "2004-09-17T17:00:00": 0.348,
    "2004-09-17T18:00:00": 0.1669,
    "2004-09-17T19:00:00": 0.17,
    "2004-09-17T20:00:00": 0.06,
    "2004-09-17T21:00:00": 0.067,
    "2004-09-17T22:00:00": 0.3959,
    "2004-09-17T23:00:00": 0.195
  },
  "148-134": {
    "2004-09-17T03:00:00": 0.0,
    "2004-09-17T04:00:00": 0.004,
    "2004-09-17T05:00:00": 0.003,
    "2004-09-17T06:00:00": 0.043,
    "2004-09-17T07:00:00": 0.137,
    "2004-09-17T08:00:00": 0.109,
    "2004-09-17T09:00:00": 0.216,
    "2004-09-17T10:

This provides a quick rainfall time-series per pixel.

---

> *EndNote: We've started codifying the above processes in a "wrapper API" available at http://3rww-rainfall-api.civicmapper.com/apidocs/, so you don't have to post-process the data like we just demonstrated. Check it out.*

## 3. Getting reference geodata

As we've seen above, 3RWW's Rainfall Data API is not spatial: it returns rainfall values for locations at points in time, but those locations are only represented by 'Pixel' IDs; it does not provide actual geometry or coordinates for those pixels.

The challenge in using that API comes in formulating the location for the request. Location is specified with a "pixel ID", which translates to a location on a 1-kilometer grid set over Allegheny County, PA. The pixel (or pixels) is a required parameter; finding and entering those raw values is somewhat tedious.

To do anything that is location specific with this data (e.g., query rainfall in a specific watershed), you'll want some geodata for reference.

### Vieux Pixel Polygons

A copy of the pixels used for all calibrated radar rainfall products (created by Vieux) are available on [3RWW's Open Data Portal](http://data-3rww.opendata.arcgis.com/) and 3RWW's regular ArcGIS Online site at:

* [data-3rww.opendata.arcgis.com/datasets/228b1584b89a45308ed4256c5bedd43d_1](https://data-3rww.opendata.arcgis.com/datasets/228b1584b89a45308ed4256c5bedd43d_1), and
* [3rww.maps.arcgis.com/home/item.html?id=228b1584b89a45308ed4256c5bedd43d](https://3rww.maps.arcgis.com/home/item.html?id=228b1584b89a45308ed4256c5bedd43d)

...respectively. We can retrieve it programmatically a couple of ways:

* with the [ArcGIS API for Python](https://developers.arcgis.com/python/); or
* by using the [Python Requests library](http://docs.python-requests.org/en/master/) to make a call directly to the Portal's ArcGIS REST API.

We'll show both ways below. 

#### Using the ArcGIS API for Python

In [10]:
# Establish a connection to your 3RWW's ArcGIS Online portal.
gis = GIS('https://3rww.maps.arcgis.com')

We can search for the feature layer by name:

In [11]:
search_results = gis.content.search('Gauge Adjusted Radar Rainfall Data')
for item in search_results:
    display(item)
garrd_item = search_results[0]

<Item title:"Gauge Adjusted Radar Rainfall Data Geometry" type:Feature Layer Collection owner:CM_3RWW>

Alternatively, we can use the item `id` to directly find the feature layer:

In [12]:
garrd_id = "228b1584b89a45308ed4256c5bedd43d"
garrd_item = gis.content.get(itemid=garrd_id)
garrd_item

<Item title:"Gauge Adjusted Radar Rainfall Data Geometry" type:Feature Layer Collection owner:CM_3RWW>

Either way gets us `gaard_item`: a feature layer *collection* item, which contains individual feature layers. This one (we know from clicking on the item above), has both points and polygons variants of the GARRD reference geometry. We're interested in the polygons (grid). Get that as follows:

In [13]:
garrd_item.layers

[<FeatureLayer url:"https://services6.arcgis.com/dMKWX9NPCcfmaZl3/arcgis/rest/services/garrd/FeatureServer/0">,
 <FeatureLayer url:"https://services6.arcgis.com/dMKWX9NPCcfmaZl3/arcgis/rest/services/garrd/FeatureServer/1">]

In [14]:
# it's the second item, index 1
garrd_grid = garrd_item.layers[1]
garrd_grid

<FeatureLayer url:"https://services6.arcgis.com/dMKWX9NPCcfmaZl3/arcgis/rest/services/garrd/FeatureServer/1">

Since we're in a notebook now, the ArcGIS API for Python lets you put that on a map:

In [15]:
m = gis.map('Pittsburgh')
m.add_layer(garrd_grid)
m

MapView(layout=Layout(height='400px', width='100%'))

Finally, we can turn that into a `geojson`-like Python dictionary.

In [16]:
q = garrd_grid.query(out_sr=4326)
garrd_grid_geojson = q.to_geojson

#### Using `requests`

This approach is a little more hands on, but works without fuss and without the overhead of the ArcGIS API for Python used above.

We need to get the service `url` from the item detail page on 3RWW's Open Data Portal, and then construct query parameters for the request as a Python dictionary.

In [17]:
# service URL - note how '/query' is at the end of the URL
service_url = 'https://services6.arcgis.com/dMKWX9NPCcfmaZl3/ArcGIS/rest/services/garrd/FeatureServer/1/query'
# query string parameters
params = {
    'where': '1=1', # Esri's convention for returning everything from the ArcGIS REST API
    'outFields': 'PIXEL', # only include the GARRD 'PIXEL' field
    'outSR': '4326', # project as WGS 1984
    'f': 'geojson' # return as geojson
}

In [18]:
# make the request
garrd_grid_geojson_response = requests.get(service_url, params=params)
garrd_grid_geojson_response

<Response [200]>

In [19]:
# this gets us the response as a geojson-like Python dictionary.
garrd_grid_geojson = garrd_grid_geojson_response.json()

That gets us a `geojson` object of all pixels as a python dictionary.

### Area of Interest Polygons

Next, we'll establish an area of interest using a polygon from an existing dataset: the Saw Mill Run watershed.

Allegheny County has a watershed dataset in ArcGIS Online, so we'll use that for this example. It's available here:

* http://openac-alcogis.opendata.arcgis.com/datasets/364f4c3613164f79a1d8c84aed6c03e0_0

(Note that you could swap this out for any online geodata service that provides polygons, and this will work)

#### With ArcGIS API for Python

In [20]:
# use the item ID from the link above to get the layer
watersheds_item = gis.content.get(itemid="364f4c3613164f79a1d8c84aed6c03e0")
watersheds_layer = watersheds_item.layers[0]
saw_mill_run = watersheds_layer.query(where="DESCR like '%SAW%'")
saw_mill_run

<FeatureSet> 1 features

In [21]:
m2 = gis.map('Pittsburgh')
m2.draw(saw_mill_run)
m2

MapView(layout=Layout(height='400px', width='100%'))

#### With `requests` library

In [22]:
# service URL - note how '/query' is at the end of the URL
service_url = 'https://services1.arcgis.com/vdNDkVykv9vEWFX4/arcgis/rest/services/Watersheds/FeatureServer/0/query'
# query string parameters
params = {
    'where': "DESCR like '%SAW MILL%'", # get Saw Mill Run
    'outFields': ['DESCR', 'AREA'], # only include the GARRD 'PIXEL' field
    'outSR': '4326', # project as WGS 1984
    'f': 'geojson' # return as geojson
}
# make the request
watershed_geojson_response = requests.get(service_url, params=params)
watershed_geojson_response

<Response [200]>

> *Note that while we're pulling our data from online sources, you could also read in your own geometry here from a shapefile on disk.*

## 4. Intersecting Pixels w/ the Area of Interest

Now that we know how to get pixel data, and know how to get area of interest data, we can perform a spatial intersection to IDs of the pixels in the area of interest, which we'll use in a query to the Teragon API.

### With the ArcGIS API for Python

Using the `garrd_grid` feature layer and the `saw_mill_run` feature_set, running and intersect is pretty easy:

In [23]:
# construct the filter using the geometry module
sa_filter = geometry.filters.intersects(geometry=saw_mill_run.features[0].geometry, sr=4326)
# then use that filter in a query of the the pixel data
pixels_of_interest = garrd_grid.query(geometry_filter=sa_filter, out_sr=4326)

In [24]:
m3 = gis.map('Pittsburgh')
m3.draw(pixels_of_interest)
m3

MapView(layout=Layout(height='400px', width='100%'))

There they are: pixels covering the Saw Mill Run watershed. Let's get a list of IDs, since that's what we're after.

First, let's introspect so we know what to go after:

In [25]:
pixels_of_interest.features[0]

{"geometry": {"rings": [[[-80.0417544856242, 40.3948600119221], [-80.0420601691609, 40.4038629358288], [-80.030283459258, 40.4040960527672], [-80.0299793449362, 40.3950930981088], [-80.0417544856242, 40.3948600119221]]], "type": "polygon", "spatialReference": {"wkid": 4326, "latestWkid": 4326}}, "attributes": {"OBJECTID": 1566, "PIXEL": "143143", "Shape__Area": 10763910.416626, "Shape__Length": 13123.3595800102}}

We can see that each Feature object is represented as a Python dictionary, and the ID is stored under `attributes` in the `PIXEL` property. We can get all the Pixel IDS out into a list with a one-liner:

In [26]:
pixel_ids = list(set([f.attributes['PIXEL'] for f in pixels_of_interest.features]))
print(pixel_ids)

['142139', '148145', '144138', '142143', '144150', '146144', '146145', '145142', '145141', '144145', '148144', '145148', '143147', '144139', '146147', '147144', '146146', '143148', '143138', '146148', '145143', '143142', '142138', '144140', '142144', '145140', '144149', '143139', '145144', '142146', '143144', '143140', '147148', '144142', '147145', '142142', '148146', '145146', '146142', '147147', '142145', '144144', '143143', '144147', '147143', '148147', '142147', '142148', '146141', '145147', '145139', '143146', '144146', '145145', '143149', '144143', '144148', '147142', '142140', '142141', '144141', '143145', '147146', '146143', '143141', '143150']


### With GeoPandas

Alternatively, we can use the raw `geojson` that we've acquired in previous steps and find the spatial intersection using the GeoPandas library.

> *To be completed*

## 5. Full Circle: getting calibrated radar rainfall data (for an area of interest)

So far you've learned how to:

* make a request to Teragon Rainfall Dataset API
* get the Pixel reference geodata
* get area of interest reference data
* find the Pixels that are in the area of interest

Now it's time to bring it all together.

### First, though...

You'll recall that the Pixel list in the Teragon API looks something like this:

`'pixels': '148,134;149,134;148,133;149,133'`.

THAT is a semi-colon delimited list of Pixel IDs as a Python `str` object; each Pixel ID is split into two 3-digit IDs (which represents non-spatial grid coordinates). Our list of Pixel IDs does not look like that!

However, we can construct that exact thing from our list above with another one-liner.

In [27]:
pixel_ids_for_api = ";".join(["{0},{1}".format(i[:3], i[-3:]) for i in pixel_ids])
pixel_ids_for_api

'142,139;148,145;144,138;142,143;144,150;146,144;146,145;145,142;145,141;144,145;148,144;145,148;143,147;144,139;146,147;147,144;146,146;143,148;143,138;146,148;145,143;143,142;142,138;144,140;142,144;145,140;144,149;143,139;145,144;142,146;143,144;143,140;147,148;144,142;147,145;142,142;148,146;145,146;146,142;147,147;142,145;144,144;143,143;144,147;147,143;148,147;142,147;142,148;146,141;145,147;145,139;143,146;144,146;145,145;143,149;144,143;144,148;147,142;142,140;142,141;144,141;143,145;147,146;146,143;143,141;143,150'

Boom. A list of Pixel IDs in the format expected by the Teragon API.

### Make the Request

Let's use that in a request to the Teragon API, just like before

In [28]:
data = {
    'startyear': 2004,
    'startmonth': 9,
    'startday': 17,
    'starthour': 3,
    'endyear': 2004,
    'endmonth': 9,
    'endday': 18,
    'endhour': 0,
    'interval': 'Hourly',
    'zerofill': 'yes',
    'pixels': pixel_ids_for_api
}
rainfall_for_saw_mill_run = requests.post(
    url="http://web.3riverswetweather.org/trp:API.pixel",
    data=data
)

And a quick reformat and print (using the helper function we defined earlier):

In [29]:
rainfall_for_saw_mill_run_clean = clean_up_response(rainfall_for_saw_mill_run.text)

etl.vis.displayall(rainfall_for_saw_mill_run_clean)

Timestamp,142-139,148-145,144-138,142-143,144-150,146-144,146-145,145-142,145-141,144-145,148-144,145-148,143-147,144-139,146-147,147-144,146-146,143-148,143-138,146-148,145-143,143-142,142-138,144-140,142-144,145-140,144-149,143-139,145-144,142-146,143-144,143-140,147-148,144-142,147-145,142-142,148-146,145-146,146-142,147-147,142-145,144-144,143-143,144-147,147-143,148-147,142-147,142-148,146-141,145-147,145-139,143-146,144-146,145-145,143-149,144-143,144-148,147-142,142-140,142-141,144-141,143-145,147-146,146-143,143-141,143-150
2004-09-17T03:00:00,0.0070,0.0050,0.0060,0.0090,0.0080,0.0080,0.0060,0.0080,0.0080,0.0070,0.0070,0.0050,0.0080,0.0080,0.0060,0.0070,0.0060,0.0070,0.0070,0.0050,0.0070,0.0070,0.0080,0.0080,0.0090,0.0090,0.0080,0.0100,0.0090,0.0070,0.0090,0.0080,0.0050,0.0060,0.0060,0.0060,0.0040,0.0050,0.0060,0.0050,0.0090,0.0070,0.0100,0.0070,0.0060,0.0060,0.0080,0.0100,0.0080,0.0080,0.0100,0.0090,0.0070,0.0060,0.0080,0.0080,0.0070,0.0050,0.0050,0.0060,0.0070,0.0080,0.0060,0.0080,0.0060,0.0110
2004-09-17T04:00:00,0.0450,0.0640,0.0480,0.0670,0.0690,0.0520,0.0530,0.0730,0.0700,0.0500,0.0560,0.0690,0.0550,0.0550,0.0630,0.0570,0.0520,0.0560,0.0440,0.0680,0.0700,0.0740,0.0430,0.0550,0.0790,0.0650,0.0670,0.0510,0.0610,0.0640,0.0730,0.0540,0.0510,0.0840,0.0520,0.0600,0.0590,0.0520,0.0610,0.0640,0.0710,0.0700,0.0760,0.0490,0.0530,0.0560,0.0550,0.0570,0.0650,0.0510,0.0570,0.0610,0.0510,0.0520,0.0610,0.0780,0.0600,0.0520,0.0470,0.0550,0.0640,0.0650,0.0640,0.0540,0.0590,0.0720
2004-09-17T05:00:00,0.0060,0.0090,0.0140,0.0090,0.0110,0.0160,0.0110,0.0120,0.0120,0.0120,0.0130,0.0140,0.0130,0.0070,0.0100,0.0150,0.0100,0.0120,0.0100,0.0100,0.0150,0.0110,0.0080,0.0100,0.0100,0.0100,0.0140,0.0090,0.0150,0.0120,0.0120,0.0090,0.0090,0.0110,0.0070,0.0100,0.0070,0.0130,0.0140,0.0090,0.0110,0.0130,0.0100,0.0120,0.0180,0.0080,0.0110,0.0130,0.0130,0.0150,0.0080,0.0110,0.0140,0.0140,0.0130,0.0130,0.0130,0.0130,0.0100,0.0120,0.0150,0.0120,0.0090,0.0150,0.0110,0.0150
2004-09-17T06:00:00,0.0390,0.0700,0.0350,0.0460,0.1130,0.0620,0.0730,0.0540,0.0480,0.0580,0.0640,0.0930,0.0750,0.0370,0.0820,0.0610,0.0690,0.0940,0.0340,0.0780,0.0560,0.0480,0.0370,0.0450,0.0480,0.0450,0.1270,0.0350,0.0650,0.0659,0.0530,0.0450,0.0720,0.0480,0.0640,0.0480,0.0730,0.0750,0.0460,0.0780,0.0560,0.0530,0.0500,0.0740,0.0510,0.0900,0.0740,0.0840,0.0470,0.0710,0.0420,0.0640,0.0650,0.0720,0.0970,0.0510,0.1000,0.0440,0.0420,0.0470,0.0420,0.0520,0.0730,0.0450,0.0430,0.1030
2004-09-17T07:00:00,0.1620,0.1460,0.1299,0.1640,0.1530,0.1699,0.1620,0.1740,0.1770,0.1700,0.1690,0.1529,0.1560,0.1590,0.1320,0.1570,0.1230,0.1480,0.1400,0.1270,0.1649,0.1620,0.1730,0.1680,0.1719,0.1650,0.1530,0.1460,0.1650,0.1779,0.1600,0.1860,0.1140,0.1900,0.1550,0.1680,0.1480,0.1550,0.1639,0.1290,0.1889,0.1460,0.1430,0.1340,0.1610,0.1560,0.1550,0.1660,0.1540,0.1260,0.1759,0.1690,0.1490,0.1710,0.1420,0.1630,0.1360,0.2040,0.1800,0.2040,0.1930,0.1839,0.1410,0.1510,0.2169,0.1370
2004-09-17T08:00:00,0.1260,0.1140,0.1130,0.1100,0.1290,0.1140,0.1070,0.1210,0.1140,0.1120,0.1240,0.1180,0.1210,0.1280,0.1020,0.1090,0.1000,0.1180,0.1160,0.1120,0.1240,0.1200,0.1350,0.1190,0.1050,0.1170,0.1419,0.1270,0.1260,0.1200,0.1100,0.1180,0.1030,0.1210,0.1090,0.1120,0.1130,0.1160,0.1100,0.0990,0.1110,0.1329,0.1090,0.1180,0.1170,0.1140,0.1260,0.1180,0.1140,0.1010,0.1210,0.1160,0.1150,0.1120,0.1270,0.1200,0.1130,0.1200,0.1209,0.1170,0.1190,0.1090,0.1110,0.0980,0.1150,0.1490
2004-09-17T09:00:00,0.1839,0.2120,0.1700,0.1930,0.2729,0.2220,0.2080,0.2190,0.2090,0.2120,0.2099,0.2150,0.1989,0.1980,0.2319,0.2040,0.2030,0.1929,0.1769,0.2290,0.2330,0.2060,0.1890,0.1919,0.2170,0.1889,0.2280,0.1850,0.2480,0.2320,0.2410,0.2030,0.2230,0.2220,0.1830,0.1929,0.1980,0.1870,0.2120,0.2180,0.2180,0.2409,0.2120,0.1760,0.2030,0.1920,0.2200,0.1790,0.2039,0.1950,0.1710,0.2129,0.1810,0.2070,0.2060,0.2290,0.1930,0.2070,0.2040,0.2000,0.1899,0.2500,0.1940,0.2160,0.1950,0.2179
2004-09-17T10:00:00,0.4809,0.3150,0.3780,0.3510,0.3440,0.3590,0.3770,0.3440,0.3640,0.407